In [1]:
import json
import csv
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import re
import sqlite3

from util import JsonLoader,DBProcer
from conf import DefaultConfig

In [2]:
#配置
cfg_c=DefaultConfig()
cfg=cfg_c.cfg
n_chara=cfg['n_chara']
date_i=cfg["date_i"]
mysql=cfg["mysql"]["config"]
halo=cfg["halo"]

col=cfg["col"]
col_c=cfg["col_c"]

In [3]:
jloader=JsonLoader(["data/f.json"],d_halo=halo,col=col,col_c=col_c)
jloader.json2csv("data/f.csv")


TypeError: list indices must be integers or slices, not NoneType

In [ ]:
#连接数据库    
db=sqlite3.connect(database=mysql["database"]+".db")
cursor=db.cursor()
table_name="records"
#sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""

'''
sql="load data local infile \"E:/cal/data/rlt.csv\" into table record character set utf8  fields terminated by ','  lines terminated by '\r\n'   ignore 1 lines;"
dbexe(db,cursor,sql)
sql="delete from record where chara=\"野\";"
dbexe(db,cursor,sql)
'''

In [1]:
!python cal.py

DB

In [ ]:
df=pd.read_csv("data/SSS.csv",encoding="utf-8")
#pd.set_option('display.max_columns',None)
name=[c for c in df.columns]
date_t=datetime.date(date_i[0],date_i[1],date_i[2])

sql="SELECT * FROM cnt;"



if rlt is not None:
    weight=dict()
    for r in rlt:
        weight[r[0]]=r[1]

# 关闭不使用的游标对象
cursor.close()
# 关闭数据库连接
db.close()

输出加权胜率

In [ ]:
weight=[float(weight[(name[2*i*(n_chara+1)].strip("\t"))[0]])/sum(weight.values()) for i in range(n_chara)]
weight=np.array(weight)

dp=len(df.index)
avg_i=dict()
avg_w=dict()
for k in range(dp):
    c_date=date_t+datetime.timedelta(k)
    c_date="{0}-{1}".format(c_date.month,c_date.day)
    avg_i[c_date]=dict()
    avg_w[c_date]=dict()
    for i in range(n_chara*2):
        di=[]
        for j in range(n_chara*i+1+i,n_chara*(i+1)+1+i):
            di.append(float(df[name[j]][k].strip('%'))/100)
        arr=np.array(di)
        avg_i[c_date][name[i*(n_chara+1)]]=np.mean(arr)
        rlt=arr*weight
        #df[name[i*(n_chara+1)]][k]=round(rlt.sum(),3)
        avg_w[c_date][name[i*(n_chara+1)]]=round(rlt.sum(),3)
#df.to_csv("out.csv",columns=[name[i*(n_chara+1)] for i in range(n_chara*2) ],encoding="gbk")
avg_w=pd.DataFrame(avg_w)
avg_w.sort_values(by=c_date,ascending=False,inplace=True)
avg_w.to_csv("data/out_w.csv",encoding="gbk")
avg_i=pd.DataFrame(avg_i)
avg_i.sort_values(by=c_date,ascending=False,inplace=True)
avg_i.to_csv("data/out_i.csv",encoding="gbk")

In [ ]:

with open("data/m.json",'r',encoding="utf-8") as f:
    datas=json.load(f)
    fcsv=[("id","enemyname","char","charlevel","time")]
    for data in datas["data"]["data"][0]["rows"]:
        fcsv.append((data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"]))
    with open("data/rlt.csv","w",newline="",encoding="utf-8") as f2:
        writer=csv.writer(f2)
        writer.writerows(fcsv)
    data=datas["data"]["data"][0]["rows"][0]

def json2csv(input,out,col=(),mode="w",encoding="utf-8"):
    with open(input,'r',encoding=encoding) as f:
        datas=json.load(f)
        if mode=="w":
            fcsv=[col]
        else:
            fcsv=[]
        for data in datas["data"]["data"][0]["rows"]:
            fcsv.append(tuple([data[c] for c in col]))
        with open(out,mode=mode,newline="",encoding=encoding) as f2:
            writer=csv.writer(f2)
            writer.writerows(fcsv)

col=("id","enemyname","char","charlevel","time")

json2csv("data/m.json","data/rlt.csv",col=col)
json2csv("data/i.json","data/rlt.csv",col=col,mode="a+")


print(data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"])
print()

In [ ]:
import yaml
import os

class DefaultConfig():
    def __init__(self,path,encoding="utf-8"):
        with open(path,"r",encoding=encoding) as f:
            self.cfg=yaml.safe_load(f)
    def loaddata(self,fname) :
        try:
            cfg_f=self.cfg["path_data"]
            path_w=cfg_f["path_data"]
            return os.path.join(path_w,cfg_f[fname])
        except ValueError as e:
            print(e)
            return

path="conf/config.yaml"
cfg=DefaultConfig(path=path)
print(cfg.loaddata("json_i"))

In [ ]:
with open("data/m.json",'r',encoding="utf-8") as f:
    datas=json.load(f)
    fcsv=[("id","enemyname","char","charlevel","time","log")]
    for data in datas["data"]["data"][0]["rows"]:
        fcsv.append((data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"],data["log"]))
    test=(fcsv[9][5])
bs = BeautifulSoup(test,"html.parser")
eq=bs.find_all(name="button",attrs={"data-original-title":True},limit=8)[4:]
for e in eq:
    print(e.attrs['data-original-title'],e.text)
tf=bs.find(name="div",attrs={"class":"col-md-7 fyg_tl"})
w_match=re.findall("\d+",tf.text)[1:]
print(w_match)
halo=bs.find(name="div",attrs={"class":"col-md-5 fyg_tr"})
w_match=re.findall(r"(?<=\|)[^|]\S{3}(?=|)",halo.text)
print(halo)
print(w_match)

In [ ]:
print(bs.prettify())

In [ ]:
str=r"""
<div class="col-md-7 fyg_tl">
      <span class="fyg_f18">
       ster1937（希 Lv.850）
      </span>
      <br/>
      [生命:762211] [护盾:10290]
      <br/>
      [速度:6801]
      <br/>
      [物攻:19899] [魔攻:12]
      <br/>
      [物防:1758] [魔防:1758]
      <br/>
     </div>
"""

w_match=re.findall("(?<=:)\d+(?=])",tf.text)
print(w_match)

line = "Cats are smarter than dogs"
 
matchObj = re.match( r'.*', line, re.M|re.I)
 
if matchObj:
   print ("matchObj.group() : ",matchObj.group())


In [ ]:
def json2csv(input,out,col=(),col_c=(),mode="w",encoding="utf-8"):
        with open(input,'r',encoding=encoding) as f:
            datas=json.load(f)
            if mode=="w":
                fcsv=[col]+[col_c]
                print(fcsv)
cfg_c=DefaultConfig()
cfg=cfg_c.cfg
n_chara=cfg['n_chara']
date_i=cfg["date_i"]
mysql=cfg["mysql"]["config"]

col=tuple(cfg["col"])
col_c=tuple(cfg["col_c"])

json2csv("data/m.json","data/rlt.csv",col=col,col_c=col_c)
json2csv("data/i.json","data/rlt.csv",col=col,col_c=col_c,mode="a+")

In [ ]:
import requests
s = requests.session()
s.keep_alive=False
url="https://www.momozhen.com/fyg_stat.php"
headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36 Edg/109.0.1518.61'}
r=requests.get(url,verify=False,headers=headers)

In [ ]:
import json
import csv
import re
from bs4 import BeautifulSoup

class JsonLoader():
    def __init__(self,inputs,d_halo,col=[],col_c=[],encoding="utf-8") -> None:
        self.encoding=encoding
        self.d_halo=d_halo
        self.col=col
        self.col_c=col_c
        self.encoding=encoding
        self.loader=[self.json_proc(inputs[0],d_halo,col,col_c,mode="w",encoding=encoding)]
        for i in range(1,len(inputs)):
            self.loader.append(self.json_proc(inputs[i],d_halo,col,col_c,mode="a+",encoding=encoding))

    def json_add(self,input):
        self.mode="a+"
        ld=self.json_proc(input,d_halo=self.d_halo,col=self.col,col_c=self.col_c,mode="a+",encoding=self.encoding)
        self.loader.append(ld)
    
    def json2csv(self,out):
        with open(out,mode="w",newline="",encoding=self.encoding) as f2:
            writer=csv.writer(f2)
            for ld in self.loader:
                writer.writerows(ld)
    def createdb(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        table_name="records"
        sql_string = f"""CREATE TABLE {table_name} ({",".join(self.loader[0][0])});"""
        cursor.execute(sql_string)
        db.commit()
        db.close()

    def json2db(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        for ld in self.loader:
            table_name="records"
            cols=self.loader[0][0]
            format_v=','.join(['?'] * len(cols))
            sql_string = f"""INSERT INTO {table_name} ({",".join(cols)}) VALUES ({format_v});""" 
            cursor.executemany(sql_string, self.loader[0][1:])
        db.commit()
        db.close()

    def json_proc(self,input,d_halo,col=[],col_c=[],mode="w",encoding="utf-8"):
        self.mode=mode
        self.encoding=encoding
        with open(input,'r',encoding=encoding) as f:
            datas=json.load(f)
            if mode=="w":
                loader=[tuple(col+col_c)]
            else:
                loader=[]
            for data in datas["data"]["data"][0]["rows"]:
                if data["char"]=="野怪":
                    pass
                else:
                    bs = BeautifulSoup(data["log"],"html.parser")
                    l_eq=bs.find_all(name="button",attrs={"data-original-title":True},limit=8)[4:]
                    eq=list()
                    for e in l_eq:
                        eq+=[e.attrs['data-original-title'],e.text ]    #装备
                    tf=bs.find(name="div",attrs={"class":"col-md-7 fyg_tl"})    #属性
                    w_match=re.findall("(?<=:)\d+(?=])",tf.text)
                    halo=bs.find(name="div",attrs={"class":"col-md-5 fyg_tr"})  #光环
                    s_halo=re.findall(r"(?<=\|)[^|]\S{3}(?=|)",halo.text)
                    bit_h=0
                    for h in s_halo:
                        bit_h+=(2**d_halo[h])
                    loader.append(tuple([data[c] for c in col]+eq+w_match+[str(s_halo)]+[bit_h]))
            return loader

if __name__=="__main__":
    from conf import DefaultConfig
    #配置
    cfg_c=DefaultConfig()
    cfg=cfg_c.cfg
    n_chara=cfg['n_chara']
    date_i=cfg["date_i"]
    mysql=cfg["mysql"]["config"]
    halo=cfg["halo"]

    col=tuple(cfg["col"])
    col_c=tuple(cfg["col_c"])
    jloader=JsonLoader(["data/m.json","data/i.json"],d_halo=halo,col=col,col_c=col_c)
    jloader.json2db()



In [ ]:

db=sqlite3.connect(database=mysql["database"]+".db")
cursor=db.cursor()
table_name="records"
sql_string = f"""SELECT * FROM {table_name} limit 5 ;"""
cursor.execute(sql_string)
d=cursor.fetchall()
print(d)
db.commit()
db.close()

In [ ]:

class JsonLoader():
    def __init__(self,inputs:list,d_halo:dict,col=[],col_c=[],encoding="utf-8") -> None:
        self.encoding=encoding
        self.d_halo=d_halo
        self.col=col
        self.col_c=col_c
        self.encoding=encoding
        self.loader=[self.json_proc(inputs[0],d_halo,col,col_c,mode="w",encoding=encoding)]
        for i in range(1,len(inputs)):
            self.loader.append(self.json_proc(inputs[i],d_halo,col,col_c,mode="a+",encoding=encoding))

    def export_loader(self) ->list:
        return self.loader

    def json_add(self,inputs:list):
        self.mode="a+"
        for i in range(len(inputs)):
            self.loader.append(self.json_proc(inputs[i],d_halo=self.d_halo,col=self.col,col_c=self.col_c,mode=self.mode,encoding=self.encoding))
    
    def json2csv(self,out:str):
        with open(out,mode="w",newline="",encoding=self.encoding) as f2:
            writer=csv.writer(f2)
            for ld in self.loader:
                writer.writerows(ld)

    def createdb(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        table_name="records"
        sql_string = f"""CREATE TABLE {table_name} ({",".join(self.loader[0][0])});"""
        cursor.execute(sql_string)
        db.commit()
        db.close()

    def json2db(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        for ld in self.loader:
            table_name="records"
            cols=self.loader[0][0]
            format_v=','.join(['?'] * len(cols))
            sql_string = f"""INSERT INTO {table_name} ({",".join(cols)}) VALUES ({format_v});""" 
            cursor.executemany(sql_string, self.loader[0][1:])
        db.commit()
        db.close()

    def json_proc(self,input:str,d_halo:dict,col=[],col_c=[],mode="w",encoding="utf-8") ->list:
        self.mode=mode
        self.encoding=encoding
        with open(input,'r',encoding=encoding) as f:
            datas=json.load(f)
            if mode=="w":
                loader=[tuple(col+col_c)]
            else:
                loader=[]
            for data in datas["data"]["data"][0]["rows"]:
                if data["char"]=="野怪":
                    pass
                else:
                    bs = BeautifulSoup(data["log"],"html.parser")
                    l_eq=bs.find_all(name="button",attrs={"data-original-title":True},limit=8)[4:]
                    eq=list()
                    for e in l_eq:
                        eq+=[e.attrs['data-original-title'],e.text ]    #装备
                    tf=bs.find(name="div",attrs={"class":"col-md-7 fyg_tl"})    #属性
                    w_match=re.findall("(?<=:)\d+(?=])",tf.text)
                    halo=bs.find(name="div",attrs={"class":"col-md-5 fyg_tr"})  #光环
                    s_halo=re.findall(r"(?<=\|)[^|]\S{3}(?=|)",halo.text)
                    bit_h=0
                    for h in s_halo:
                        bit_h+=(2**d_halo[h])
                    loader.append(tuple([data[c] for c in col]+eq+w_match+[str(s_halo)]+[bit_h]))
            return loader



class DBProcer():
    def __init__(self,path_db:str) -> None:
        self.path_db=path_db

    def dbexe(self,sql_string:str) -> list:
        db=sqlite3.connect(database=self.path_db)
        cursor=db.cursor()
        cursor.execute(sql_string)
        db.commit()
        db.close()

    def create_table(self,table_name:str,cols:list):
        sql_string = f"""CREATE TABLE IF NOT EXISTS {table_name} ({",".join(cols)});"""
        self.dbexe(sql_string)

    def create_view(self,view_name:str,table_name:str,day:int,cols:list):
        cols_s=",".join(cols)
        sql_string=f"""CREATE VIEW IF NOT EXISTS {view_name} as select enemyname,char,charlevel from (select * from {table_name} where (1674662400000-time)<{str(day*100000000)} order by time desc limit 100000) as v group by v.enemyname;"""
        self.dbexe(sql_string)

    def droper(self,type,name):
        sql_string=f"""DROP {type} IF EXISTS {name};"""
        self.dbexe(sql_string)
    
    def json2db(self,table_name,loader:list):
        cols=loader[0][0]
        self.cols=cols
        self.create_table(table_name=table_name,cols=cols)
        db=sqlite3.connect(database=self.path_db)
        cursor=db.cursor()
        for ld in loader:
            format_v=','.join(['?'] * len(cols))
            sql_string = f"""INSERT INTO {table_name} ({",".join(cols)}) VALUES ({format_v});""" 
            cursor.executemany(sql_string, ld[1:])
        db.commit()
        db.close()

jloader=JsonLoader(["data/f.json"],d_halo=halo,col=col,col_c=col_c)
loader=jloader.export_loader()
dbs=DBProcer("ggz.db")
dbs.create_view("player","t2",42,[])
#dbs.droper("view","player")

In [8]:
dbs=DBProcer("ggz.db")
dbs.create_view("player","records",42,["enemyname","char","charlevel"])
#dbs.droper("view","cnt")
dbs.dbexe(f"create view cnt as select char \"chara\",count(*) \"num\" from player group by player.char;")

In [44]:
bit=2005696
print( bit&2<<(halo["沸血之志"]-2))

print(2<<(halo["沸血之志"]-2))

1048576
1048576


In [2]:
import json
import csv
import re
from bs4 import BeautifulSoup
import sqlite3
from util import JsonLoader,DBProcer


if __name__=="__main__":
    from conf import DefaultConfig
    #配置
    cfg_c=DefaultConfig()
    cfg=cfg_c.cfg
    n_chara=cfg['n_chara']
    date_i=cfg["date_i"]
    mysql=cfg["mysql"]["config"]
    halo=cfg["halo"]

    col=cfg["col"]
    col_c=cfg["col_c"]

    jloader=JsonLoader(["data/i.json","data/m.json"],d_halo=halo,col=col,col_c=col_c)
    loader=jloader.export_loader()
    jloader.json2csv("rlt2.csv")
    from util import DBProcer
    dbs=DBProcer("ggz.db")
    #dbs.droper("table","rcd")
    dbs.json2db("records",loader)
    #dbs.create_view("player","records",42,["enemyname","char","charlevel"])
    #dbs.droper("view","cnt")
    dbs.dbexe(f"create view cnt as select char \"chara\",count(*) \"num\" from player group by player.char order by num DESC;")   


In [3]:
dbs.droper("view","cnt")
dbs.dbexe(f"create view cnt as select char \"chara\",count(*) \"num\" from player group by player.char order by num DESC;")   